<a href="https://colab.research.google.com/github/leahandofir/earthformer-inference-experiments/blob/main/earthformer_SEVIR_train_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trying to run the training code on a small dataset (https://github.com/amazon-science/earth-forecasting-transformer/tree/main/scripts/cuboid_transformer/sevir)

Install dependecies

In [ ]:
# install CUDA 11.6.2 https://stackoverflow.com/questions/50560395/how-to-install-cuda-in-google-colab-gpus
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
!mv cuda-ubuntu2004.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.6.2/local_installers/cuda-repo-ubuntu2004-11-6-local_11.6.2-510.47.03-1_amd64.deb
!dpkg -i cuda-repo-ubuntu2004-11-6-local_11.6.2-510.47.0-1_amd64.deb
!apt-key add /var/cuda-repo-ubuntu2004-11-6-local/7fa2af80.pub
!apt-get update
!apt-get -y install cuda-11-6

In [23]:
# symlink to be the current CUDA
!rm /usr/local/cuda
!ln -s /usr/local/cuda-11.6 /usr/local/cuda

In [ ]:
!nvcc --version

In [24]:
%%shell
# install dependencies
pip install --upgrade pip
pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 torchaudio==0.12.1 torchtext==0.13.1 torchdata==0.4.1 --extra-index-url https://download.pytorch.org/whl/cu116
python3 -m pip install "pytorch_lightning>=1.6.4,<1.8.0"
python3 -m pip install xarray netcdf4 opencv-python
git clone https://github.com/amazon-science/earth-forecasting-transformer.git
cd earth-forecasting-transformer
python3 -m pip install -U -e . --no-build-isolation
pip install -v --disable-pip-version-check --no-cache-dir pytorch-extension git+https://github.com/NVIDIA/apex.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'earth-forecasting-transformer' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/earth-forecasting-transformer/scripts/cuboid_transformer/sevir/earth-forecasting-transformer
  Preparing metadata (setup.py) ... done
  Attempting uninstall: earthformer
    Found existing installation: earthformer 0.0.1.dev20230514
    Uninstalling earthformer-0.0.1.dev20230514:
      Successfully uninstalled earthformer-0.0.1.dev2023

In [25]:
# !curl 'https://raw.githubusercontent.com/leahandofir/earthformer-inference-experiments/main/prerequisites.sh' -o ./prerequisites.sh
# !bash ./prerequisites.sh
%env HOME=/content/earth-forecasting-transformer
%cd ~

env: HOME=/content/earth-forecasting-transformer
/content/earth-forecasting-transformer


In [29]:
# download all 2019 data
!aws s3 cp --no-sign-request --recursive --region us-west-2 s3://sevir/data/vil/2019 ~/datasets/sevir/data/vil/2019
!aws s3 cp --no-sign-request --region us-west-2 s3://sevir/CATALOG.csv ~/datasets/sevir/CATALOG.csv

download: s3://sevir/data/vil/2019/SEVIR_VIL_STORMEVENTS_2019_0701_1231.h5 to ../../../datasets/sevir/data/vil/2019/SEVIR_VIL_STORMEVENTS_2019_0701_1231.h5
download: s3://sevir/data/vil/2019/SEVIR_VIL_STORMEVENTS_2019_0101_0630.h5 to ../../../datasets/sevir/data/vil/2019/SEVIR_VIL_STORMEVENTS_2019_0101_0630.h5
download: s3://sevir/data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0901_1231.h5 to ../../../datasets/sevir/data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0901_1231.h5
download: s3://sevir/data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0101_0430.h5 to ../../../datasets/sevir/data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0101_0430.h5
download: s3://sevir/data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0501_0831.h5 to ../../../datasets/sevir/data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0501_0831.h5
download: s3://sevir/CATALOG.csv to ../../../datasets/sevir/CATALOG.csv


In [38]:
%cd scripts/cuboid_transformer/sevir

/content/earth-forecasting-transformer/scripts/cuboid_transformer/sevir


In [39]:
from omegaconf import OmegaConf
conf = OmegaConf.load('./cfg_sevir.yaml')
conf.dataset.start_date = [2019, 1, 1]
conf.dataset.end_date = [2019, 12, 1]
conf.dataset.train_val_split_date = [2019, 1, 31]
conf.dataset.train_test_split_date = [2019, 5, 1]
OmegaConf.save(config=conf, f='./my_cfg_sevir.yaml')

In [40]:
!MASTER_ADDR=localhost MASTER_PORT=10001 python train_cuboid_sevir.py --gpus 1 --cfg my_cfg_sevir.yaml --ckpt_name last.ckpt --save tmp_sevir

Global seed set to 0
Traceback (most recent call last):
  File "/content/earth-forecasting-transformer/scripts/cuboid_transformer/sevir/train_cuboid_sevir.py", line 792, in <module>
    main()
  File "/content/earth-forecasting-transformer/scripts/cuboid_transformer/sevir/train_cuboid_sevir.py", line 740, in main
    dm.setup()
  File "/content/earth-forecasting-transformer/scripts/cuboid_transformer/sevir/earth-forecasting-transformer/src/earthformer/datasets/sevir/sevir_torch_wrap.py", line 222, in setup
    self.sevir_train = SEVIRTorchDataset(
  File "/content/earth-forecasting-transformer/scripts/cuboid_transformer/sevir/earth-forecasting-transformer/src/earthformer/datasets/sevir/sevir_torch_wrap.py", line 39, in __init__
    self.sevir_dataloader = SEVIRDataLoader(
  File "/content/earth-forecasting-transformer/scripts/cuboid_transformer/sevir/earth-forecasting-transformer/src/earthformer/datasets/sevir/sevir_dataloader.py", line 225, in __init__
    self._open_files(verbose=sel